In [1]:
!pip install langchain_openai langchain_community pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.6 MB/s eta 0:00:

# 1) 환경설정 및 데이터 셋 로드

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr


In [4]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")

('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x7d7ab737d450>)

In [13]:
loader =PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print("문서의 수:", len(texts))

문서의 수: 366


In [14]:
texts[14]

Document(metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 18, 'page_label': '19'}, page_content='2\n경제금융용어 700선\n흑자를 냈다면 그 가정은 벌어들인 수입 일부만을 사용했다는 것을 의미하며, 적자를 \n냈다면 수입 외에 빚을 추가로 얻어 사용한 것이라고 보아야 한다. 우리나라는 통계청에\n서 가계의 수입과 지출을 조사하여 국민의 소득수준 및 생활실태를 파악하기 위해 \n표본으로 선정된 가계에 가계부를 나누어 주고 한 달간의 소득과 지출을 기록하도록 \n한 다음 이를 토대로 가계수지 통계를 작성하여 발표하고 있다. 가계부의 소득항목에는 \n근로소득･사업소득･재산소득･이전소득 항목이 있고, 비용항목에는 식료품비･주거비･\n수도광열비･보건의료비･교육비 항목이 있다 .\n 연관검색어 : 경상수지, 재정수지 \n가계순저축률\n일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로 \n가계순저축률은 가계부문의 순저축액을 가계순처분가능소득과 정부로부터 받은 사회적 \n현물이전 금액, 연금기금의 가계순지분 증감조정액을 합계한 금액으로 나눈 비율이다. \n여기서 사회적 현물이전(socia

In [15]:
print(texts[14].page_content)

2
경제금융용어 700선
흑자를 냈다면 그 가정은 벌어들인 수입 일부만을 사용했다는 것을 의미하며, 적자를 
냈다면 수입 외에 빚을 추가로 얻어 사용한 것이라고 보아야 한다. 우리나라는 통계청에
서 가계의 수입과 지출을 조사하여 국민의 소득수준 및 생활실태를 파악하기 위해 
표본으로 선정된 가계에 가계부를 나누어 주고 한 달간의 소득과 지출을 기록하도록 
한 다음 이를 토대로 가계수지 통계를 작성하여 발표하고 있다. 가계부의 소득항목에는 
근로소득･사업소득･재산소득･이전소득 항목이 있고, 비용항목에는 식료품비･주거비･
수도광열비･보건의료비･교육비 항목이 있다 .
 연관검색어 : 경상수지, 재정수지 
가계순저축률
일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로 
가계순저축률은 가계부문의 순저축액을 가계순처분가능소득과 정부로부터 받은 사회적 
현물이전 금액, 연금기금의 가계순지분 증감조정액을 합계한 금액으로 나눈 비율이다. 
여기서 사회적 현물이전(social transfer in kind)이란 정부 등이 가계에 현물이전의 
형태로 제공하는 재화 및 서비스로서 무상교육, 보건소의 무상진료 등이 해당된다 . 
또한 연금기금의 가계순지분 증감조정액을 분모에 더하는 이유는 퇴직연금 등과 같이 
가계가 납부한 연금부담금과 연금수취액의 차액을 반영해야 가계부문의 저축액을 정확
히 구할 수 있기 때문이다. 가계순저축률은 가계부문의 저축성향을 가장 잘 나타내주는 
지표라고 할 수 있다 .
가계순저축률  가계부문순저축 가계순처분가능소득 사회적현물이전수취
연금기금의가계순지분증감조정 ×
 연관검색어 : 가계처분가능소득, 저축률/총저축/평균소비성향/ 평균저축성향
가계신용통계
가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은 
금융기관뿐 아니라 정부, 판매회사 등 기타기관이 가계에 제공한 대출과 외상구매 관련


# 2) 데이터셋 전처리

In [16]:
#머리말
print(texts[0].page_content)

iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말


In [17]:
#목차
print(texts[5].page_content)

vi
경제금융용어 700선
ㅂ
바이오인증 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
바젤은행감독위원회/
바젤위원회(BCBS) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
발행시장 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・119
발행중지화폐/유통정지화폐 ・・・・・・・・・・・・・・・120
방카슈랑스 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・120
배당할인모형 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
밴드웨건효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
뱅 크 런 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・122
범위의 경제 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
법률리스크 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
베블런효과 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・123
변동금리 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・123
변동금리부채권(FRN) ・ ・ ・・・・・・・・・・・・・・・・・・・・・・・123
보기화폐(견양화폐) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・124
보완자본(Tier 2) ・・・・・・・・・・・

In [18]:
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [19]:
#0번부터 12번 texts 삭제, 마지막 청크 삭제
texts = texts[13:-1]
print(len(texts))

352


# 3) 임베딩 및 벡터 데이터베이스 적재

In [20]:
embedding = OpenAIEmbeddings(chunk_size = 100)
#객체 생성
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding
)

In [22]:
print(vectordb._collection.count())

352


In [23]:
for key in vectordb._collection.get():
    print(key)

ids
embeddings
documents
uris
included
data
metadatas


In [24]:
documents = vectordb._collection.get()['documents']
print("청크의 개수:", len(documents))
print("0번 청크:", documents[0])

청크의 개수: 352
0번 청크: 1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [26]:
#임베딩 벡터 값
embeddings = vectordb._collection.get(include = ['embeddings'])['embeddings']
print("임베딩 벡터 수:", len(embeddings))
print("0번 청크의 임베딩 값:", embeddings[0])
print("0번 청크의 임베딩 값의 길이:", len(embeddings[0]))

임베딩 벡터 수: 352
0번 청크의 임베딩 값: [-0.0077088  -0.0075166   0.014182   ... -0.01861645 -0.00669286
 -0.00903364]
0번 청크의 임베딩 값의 길이: 1536


In [29]:
#메타데이터(출처) 호출
metadatas = vectordb._collection.get()['metadatas']
print("metadatas의 개수:", len(metadatas))
print("0번 청크의 출처:", metadatas[0])

metadatas의 개수: 352
0번 청크의 출처: {'creationdate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'page': 17, 'creator': 'Smart PDF Imposition 1.0', 'producer': 'Smart PDF Imposition 1.0', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'author': 'INSUNG DATA INC.', 'page_label': '18', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'moddate': 'January 31, 2018 16:21', 'creationdate--text': 'January 31, 2018 16:21'}




*   as_retriever(): 입력된 텍스트로부터 유사한 텍스트 찾음
*   get_relevant_documents(): 입력된 텍스트와 유사한 청크 찾음



In [31]:
retriever = vectordb.as_retriever(search_kwargs = {'k':2}) # 유사도 순위 지정
docs = retriever.get_relevant_documents("비트코인이 궁금해")

print("유사 문서 개수:", len(docs))
print("첫번째 유사 문서:", docs[0])
print("--" * 50)
print("두번째 유사 문서:", docs[1])

유사 문서 개수: 2
첫번째 유사 문서: page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다른 
속성을 지닌다. 한편 2017년 8월 1일 비트코인(classic bit

# 4) 프롬프트 템플릿, LLM, 리트리버의 연결

In [32]:
template = """ 당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다.
안상준 개발자가 만들었습니다. 주어진 검색결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요, 반말로 친근하게 답변하세요.
{context}

Question : {question}
Answer:
"""
prompt = PromptTemplate.from_template(template)

In [37]:
#객체 생성
llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0)

In [38]:
#LLM 객체, 프롬프트 템플릿, 검색기 객체 retriever 연결

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type_kwargs = {'prompt':prompt},
    retriever = retriever,
    return_source_documents = True
)

In [39]:
input_text = "디커플링이란 무엇인가?"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '디커플링이란 무엇인가?', 'result': '디커플링은 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 말해. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 미국 경제와 다른 모습을 보이는 경우가 있어. 주가나 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 흐름을 보이는 것도 디커플링이라고 할 수 있어. 반대로, 경제가 다른 국가나 세계 경제 흐름과 유사한 흐름을 보이는 것을 커플링이라고 해.', 'source_documents': [Document(metadata={'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'moddate--text': 'January 31, 2018 16:21', 'creationdate': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'moddate': 'January 31, 2018 16:21', 'page_label': '115', 'creationdate--text': 'January 31, 2018 16:21', 'source': '2020_경제금융용어 700선_게시.pdf', 'author': 'INSUNG DATA INC.', 'total_pages': 371, 'creator': 'Smart PDF Imposition 1.0', 'page': 114}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통화정책\n디커플링/커플링\n디커플링(

In [40]:
def get_chatbot_response(input_text):
    chatbot_response = qa_chain(input_text)
    return chatbot_response['result'].strip()

In [41]:
input_text = "넌 뭘하는 챗봇이니?"
result = get_chatbot_response(input_text)
print(result)

난 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 안상준 개발자가 만들었고, 주어진 검색결과를 바탕으로 금융 용어에 대해 설명해줄 수 있어. 궁금한 게 있으면 물어봐!


In [42]:
input_text = "비트코인에 대해서 궁금하당!"
result = get_chatbot_response(input_text)
print(result)

비트코인은 가상통화이자 디지털 지급 시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화야. 사토시 나카모토라는 사람(또는 집단)이 2009년에 만들었고, 블록체인 기술을 기반으로 하고 있어. 거래는 중개자 없이 참여자들 사이에서 직접 이루어지고, 작업증명을 통해 검증된 후 블록체인에 기록돼. 비트코인은 채굴이라는 과정을 통해 발행되고, 다른 통화나 상품, 서비스와 교환될 수 있어. 하지만 가격 변동이 심하고, 익명성 때문에 불법 거래에 악용될 가능성도 있어서 일부 국가에서는 규제를 하고 있어. 비트코인의 총량은 2100만 개로 한정되어 있고, 2140년경 모두 채굴될 것으로 예상돼.


# 5) 그라디오 웹 인터페이스

In [ ]:
#인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label = "경제금융용어 챗봇")
    msg = gr.Textbox(label = "질문해주세요!")
    clear = gr.Button("대화 초기화")

    #챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
        result= qa_chain(message)
        bot_message = result['result']

        #채팅 기록에 사용자의 메시지와 봇의 응답을 추가
        chat_history.append((message ,bot_message))
        return "",chat_history

    #사용자의 입력을 제출하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    #초기화 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: None, None, chatbot, queue = False)

#인터페이스 실행
demo.launch(debug = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e3ea89350477381754.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
